In [ ]:
import requests
import pandas as pd
from matplotlib import pyplot as plt
from common import *


In [ ]:
r = requests.get('https://nvebiapi.nve.no/api/Magasinstatistikk/HentOffentligData')
x = r.json()
df = pd.DataFrame(x)
df = df.sort_values(by='dato_Id', ascending=True, ignore_index=True)
#df.to_csv(f"{INPUT_DATA}magasin_statistikk_nve.csv")
df.head()

In [ ]:
def get_område(df, omrnr, data_from_year=2016):
    omr = df.loc[df['omrnr'] == omrnr]
    omr = omr.loc[omr['iso_aar'] >= data_from_year]
    omr = omr.loc[omr['omrType'] == "EL"]
    return omr

In [ ]:
ax = get_område(df, 1).plot("dato_Id", "fyllingsgrad", figsize=(20,8))

get_område(df, 2).plot("dato_Id", "fyllingsgrad", figsize=(20,8), ax=ax)
get_område(df, 3).plot("dato_Id", "fyllingsgrad", figsize=(20,8), ax=ax)
get_område(df, 4).plot("dato_Id", "fyllingsgrad", figsize=(20,8), ax=ax)
get_område(df, 5).plot("dato_Id", "fyllingsgrad", figsize=(20,8), ax=ax)
plt.title("Fyllingsgrad i %")
plt.ylabel("TWh")
plt.xlabel("Date")


In [ ]:
ax = get_område(df, 1).plot("dato_Id", "fylling_TWh", figsize=(20,8), label="NO1")
get_område(df, 2).plot("dato_Id", "fylling_TWh", figsize=(20,8), ax=ax, label="NO2")
get_område(df, 3).plot("dato_Id", "fylling_TWh", figsize=(20,8), ax=ax, label="NO3")
get_område(df, 4).plot("dato_Id", "fylling_TWh", figsize=(20,8), ax=ax, label="NO4")
get_område(df, 5).plot("dato_Id", "fylling_TWh", figsize=(20,8), ax=ax, label="NO5")
plt.title("Fylling TWh")
plt.ylabel("TWh")
plt.xlabel("Date")

In [ ]:
get_område(df, 2).plot("dato_Id", "fyllingsgrad", figsize=(20,8))

In [ ]:
get_område(df, 3).plot("dato_Id", "fyllingsgrad", figsize=(20,8))

In [ ]:
get_område(df, 4).plot("dato_Id", "fyllingsgrad", figsize=(20,8))

In [ ]:
get_område(df, 5).plot("dato_Id", "fyllingsgrad", figsize=(20,8))

## CREATE DATASET FOR TRAINING:  

In [ ]:

def get_fyllingsgrad_array(omrnr:int, data_from_year:int)->pd.DataFrame:   
    df['year'] = pd.DatetimeIndex(df['dato_Id']).year
    omr = df.loc[df['omrnr'] == omrnr]
    omr = omr.loc[omr['year'] >= data_from_year]
    omr = omr.loc[omr['omrType'] == "EL"]
    omr = omr["fyllingsgrad"]
    return omr.to_numpy()




In [ ]:
for omrnr in range(1,6):
     #init empty array with nans   
     magasin_stats = np.empty(len(df_entso))
     magasin_stats[:] = np.nan
     fyllingsgrad_array = get_fyllingsgrad_array(omrnr,2016)
     #fill array with 1 weekly magazine-stat, the rest of the week as NaN:
     nve_time_step = 0
     for timestep in range(0,len(magasin_stats),(24*7)):
          magasin_stats[timestep] = fyllingsgrad_array[nve_time_step]
          nve_time_step+=1
     #Add stats to entso:
     df_entso[f"NO{omrnr}_Fyllingsgrad_NVE"] = magasin_stats

     print(f"Number of Non-Nan values after filling: {np.count_nonzero(~np.isnan(magasin_stats))}")
     
#Save to csv:      
#df_entso.to_csv(f"{INPUT_DATA}entso_NVE.csv")
     

### Interpolate Nan: 

In [ ]:
for omrnr in range(1,6):
    df_entso[f"NO{omrnr}_Fyllingsgrad_NVE"].interpolate(method= "linear", inplace= True)
#Save to csv:      
#df_entso.to_csv(f"{INPUT_DATA}entso_NVE_interpolated.csv")

In [ ]:
df_entso.head()